In [ ]:
import math
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
def download_stock_data(ticker, start_date, end_date):
    return yf.download(ticker, start=start_date, end=end_date)

def preprocess_data(data):
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = scaler.fit_transform(data)
    return scaled_data

In [ ]:
start_date = '2022-01-01'
end_date = '2024-03-12'

df = download_stock_data('PPL.KA', start_date, end_date)

data = df.filter(['Close']).values
data = data.reshape(-1, 1)
scaled_data = preprocess_data(data)

[*********************100%%**********************]  1 of 1 completed


In [ ]:
# prompt: scale all of these columns and create a df Open	High	Low	Close	Adj Close	Volume

df_scaled = df.copy()
for column in ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']:
    df_scaled[column] = preprocess_data(df[column].values.reshape(-1, 1))


In [ ]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,79.250000,79.790001,78.599998,78.940002,72.193100,451370
2022-01-04,79.190002,80.150002,79.099998,79.620003,72.814980,923790
2022-01-05,79.510002,80.199997,79.000000,79.059998,72.302849,902026
2022-01-06,79.379997,79.379997,78.010002,78.180000,71.498055,465370
2022-01-07,78.870003,78.870003,78.029999,78.589996,71.873016,307563
...,...,...,...,...,...,...
2024-03-05,118.000000,118.989998,113.300003,114.199997,113.266075,6853016
2024-03-06,113.809998,117.000000,113.809998,114.349998,113.414848,7916062
2024-03-07,114.050003,115.980003,113.519997,115.300003,114.357086,4384060


In [ ]:
df_scaled

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,0.340120,0.323752,0.344503,0.337080,0.295641,0.012778
2022-01-04,0.339398,0.327941,0.350618,0.345299,0.303228,0.026151
2022-01-05,0.343253,0.328523,0.349395,0.338530,0.296980,0.025535
2022-01-06,0.341687,0.318981,0.337288,0.327895,0.287162,0.013174
2022-01-07,0.335542,0.313046,0.337532,0.332850,0.291736,0.008707
...,...,...,...,...,...,...
2024-03-05,0.806988,0.779937,0.768864,0.763234,0.796723,0.194001
2024-03-06,0.756506,0.756779,0.775101,0.765047,0.798538,0.224095
2024-03-07,0.759398,0.744909,0.771554,0.776529,0.810033,0.124108


In [ ]:
def create_sequences(data, seq_len):
    x = []
    y = []
    for i in range(seq_len, len(data)):
        x.append(data[i-seq_len:i, 0])
        y.append(data[i, 0])
    return np.array(x), np.array(y)

def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))
    return model

In [ ]:
def train_model(model, x_train, y_train, batch_size, epochs):
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)
    return model

def test_model(model, x_test, scaler):
    predictions = model.predict(x_test)
    predictions = scaler.inverse_transform(predictions)
    return predictions

In [ ]:
def visualize_results(train_data, valid_data, predictions):
    plt.figure(figsize=(16,8))
    plt.title('Model')
    plt.xlabel('Date', fontsize=18)
    plt.ylabel('Close Price USD ($)', fontsize=18)
    plt.plot(train_data['Close'])
    plt.plot(valid_data[['Close', 'Predictions']])
    plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
    plt.show()

In [ ]:
symbols = ['PPL.KA', 'ACPL.KA', 'PSO.KA']
start_date = '2022-01-01'
end_date = '2024-03-12'


In [ ]:
symbols = ['PPL.KA', 'ACPL.KA', 'PSO.KA']
start_date = '2022-01-01'
end_date = '2024-03-12'

data = download_stock_data('PPL.KA', start_date, end_date)

data = data.filter(['Close']).values
data = data.reshape(-1, 1)
preprocess_data(data)

[*********************100%%**********************]  1 of 1 completed


array([[0.33708007],
       [0.34529859],
       [0.33853034],
       [0.32789465],
       [0.33284988],
       [0.33188307],
       [0.32620261],
       [0.33611317],
       [0.33478369],
       [0.34529859],
       [0.3515833 ],
       [0.34384823],
       [0.35496738],
       [0.35218764],
       [0.35436313],
       [0.34529859],
       [0.34022238],
       [0.33562971],
       [0.33284988],
       [0.34517771],
       [0.35859323],
       [0.37962294],
       [0.36910804],
       [0.36656998],
       [0.38288623],
       [0.37466771],
       [0.38578685],
       [0.37756833],
       [0.37490939],
       [0.37237133],
       [0.35738465],
       [0.35339623],
       [0.3515833 ],
       [0.36560317],
       [0.34759488],
       [0.34747409],
       [0.32813633],
       [0.30928211],
       [0.32741119],
       [0.38856659],
       [0.40089442],
       [0.44065755],
       [0.43546054],
       [0.40524541],
       [0.3779309 ],
       [0.36693264],
       [0.37660143],
       [0.361

In [ ]:
data = df.filter(['Close']).values
data = data.reshape(-1, 1)
preprocess_data(data)

array([[0.33708007],
       [0.34529859],
       [0.33853034],
       [0.32789465],
       [0.33284988],
       [0.33188307],
       [0.32620261],
       [0.33611317],
       [0.33478369],
       [0.34529859],
       [0.3515833 ],
       [0.34384823],
       [0.35496738],
       [0.35218764],
       [0.35436313],
       [0.34529859],
       [0.34022238],
       [0.33562971],
       [0.33284988],
       [0.34517771],
       [0.35859323],
       [0.37962294],
       [0.36910804],
       [0.36656998],
       [0.38288623],
       [0.37466771],
       [0.38578685],
       [0.37756833],
       [0.37490939],
       [0.37237133],
       [0.35738465],
       [0.35339623],
       [0.3515833 ],
       [0.36560317],
       [0.34759488],
       [0.34747409],
       [0.32813633],
       [0.30928211],
       [0.32741119],
       [0.38856659],
       [0.40089442],
       [0.44065755],
       [0.43546054],
       [0.40524541],
       [0.3779309 ],
       [0.36693264],
       [0.37660143],
       [0.361

In [ ]:
for symbol in symbols:
    df = download_stock_data(symbol, start_date, end_date)
    data = df.filter(['Close']).values
    data = data.reshape(-1, 1)  # Reshape data to be 2D
    scaler = preprocess_data(data)
    x_train, y_train = create_sequences(scaled_data, seq_len=60)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    model = build_lstm_model((x_train.shape[1], 1))
    model = train_model(model, x_train, y_train, batch_size=1, epochs=1)
    test_data = scaled_data[-60:]  # Get last 60 elements
    test_data = test_data.reshape(1, -1, 1)  # Reshape test data to be 3D
    predictions = test_model(model, test_data, scaler)
    pred_dates = pd.date_range(start=end_date, periods=len(predictions))
    predictions_df = pd.DataFrame(predictions, index=pred_dates, columns=['Predictions'])
    df = pd.concat([df, predictions_df], axis=1)
    # visualize_results(df[:len(data)-60], df[len(data)-60:], symbol)

In [ ]:
df

,Open,High,Low,Close,Adj Close,Volume,Predictions
Date,,,,,,,
2022-01-03,79.250000,79.790001,78.599998,78.940002,72.193100,451370,79.366599
2022-01-04,79.190002,80.150002,79.099998,79.620003,72.814980,923790,80.046599
2022-01-05,79.510002,80.199997,79.000000,79.059998,72.302849,902026,79.486594
2022-01-06,79.379997,79.379997,78.010002,78.180000,71.498055,465370,78.606597
2022-01-07,78.870003,78.870003,78.029999,78.589996,71.873016,307563,79.016593
...,...,...,...,...,...,...,...
2024-03-05,118.000000,118.989998,113.300003,114.199997,113.266075,6853016,114.626593
2024-03-06,113.809998,117.000000,113.809998,114.349998,113.414848,7916062,114.776595
2024-03-07,114.050003,115.980003,113.519997,115.300003,114.357086,4384060,115.726600
